## Comprehensive Portfolio Optimization and Visualization

This notebook analyzes and optimizes a portfolio of selected assets. It includes the following:

1. **Data Analysis**: Download stock prices and calculate correlation.
2. **Portfolio Metrics**: Compute volatility, return, and Sharpe Ratio.
3. **Optimization**: Find the Global Minimum Variance Portfolio (GMVP) and Maximum Sharpe Ratio Portfolio (MSR).
4. **Performance Evaluation**: Use metrics like Jensen's Alpha, Treynor Ratio, and Sharpe Ratio.
5. **Visualizations**:
   - Correlation Heatmap.
   - Portfolio weights for GMVP and MSR.
   - Efficient Frontier and Capital Market Line (CML).
   - Security Market Line (SML) with dynamic GMVP.
   - Fama-French 3-Factor Model.

## Selected Assets

The portfolio consists of the following assets:
- **Michelin (ML.PA)**: Automotive (Tires).
- **HDFC Bank (HDB)**: Financial Services (India).
- **Constellation Energy (CEG)**: Energy (Nuclear Power).
- **Raytheon Technologies (RTX)**: Aerospace and Defense.
- **Mitsui & Co. (8031.T)**: Diversified Conglomerate (Japan).
te (Japan).
 Conglomerate (Japan).



## Code Summary: - Import Libraries

This code sets up the necessary libraries for financial analysis, data manipulation, optimization, and visualization.

### 1. Financial Data Retrieval
- **`yfinance`**: Downloads financial data (e.g., stock prices, indices).
- **`pandas_datareader`**: Alternative for retrieving financial data from sources like Yahoo Finance.

### 2. Data Manipulation and Numerical Calculations
- **`numpy`**: Handles multidimensional arrays and mathematical operations (e.g., mean, variance).
- **`pandas`**: Processes and analyzes tabular data (e.g., stock price time series).

### 3. Date Management
- **`datetime`**: Manages date and time operations (e.g., defining analysis periods).

### 4. Optimization and Advanced Statistics
- **`scipy.optimize`**: Solves optimization problems (e.g., portfolio Sharpe Ratio maximization).
- **`statsmodels`**: Provides statistical tools (e.g., regression analysis for Fama-French models).

### 5. Data Visualization
- **`matplotlib.pyplot`**: Creates basic plots (e.g., Efficient Frontier).
- **`seaborn`**: Simplifies creation of advanced visualizations (e.g., correlation heatmaps).

### 6. Performance and Warning Management
- **`time`**: Measures execution time for performance evaluation.
- **`warnings`**: Suppresses unnecessary warnings (e.g., runtime or future warnings).

### 7. Logging and File Management
- **`logging`**: Provides a flexible framework for logging events and errors in your code.
- **`os`**: Offers functions for interacting with the operating system (e.g., file path operations).

### 8. JSON Handling
- **`json`**: Allows for parsing and writing JSON data.

### 9. Excel File Creation
- **`xlsxwriter`**: Used for writing data to Excel files and creating complex spreadsheets.

### 10. Additional Libraries for Data Retrieval
- **`pandas_datareader`**: Used for retrieving data from online sources like Yahoo Finance.
- **`xgboost`**: For advanced machine learning (optional, for predictive analysis).

## Purpose
This setup enables efficient financial portfolio analysis, including:
- Data retrieval and processing.
- Portfolio optimization.
- Visualization of results.

## For more details, refer to the "requirements"


In [3]:
import logging
import yfinance as yf
import numpy as np
import pandas as pd
import os
import json
from datetime import datetime
import scipy.optimize as sco
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from pandas_datareader import data as pdr
import warnings
import xlsxwriter

warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Code Explanation: - Constants

This code block defines important constants that are essential for the financial analysis program.

## Configuration

Before beginning you have to configurate the code with your data on this template:



    "tickers": ["ML.PA", "HDB", "CEG", "RTX", "8031.T"],
    "start_date": "2019-01-01",
    "end_date": null,
    "risk_free_rate": 0.05,
    "output_path": "./plots/",
    "base_currency": "USD"


    
## 1. Asset Tickers
The tickers represent the financial assets included in the portfolio. Each ticker is unique to a specific stock or company and is used to fetch data and perform analyses.

### Ticker Details:
- **`ML.PA`**: Represents **Michelin**, a French multinational tire manufacturer, listed on the **Paris Stock Exchange**.
- **`HDB`**: Stands for **HDFC Bank**, a major financial services provider in **India**.
- **`CEG`**: Refers to **Constellation Energy**, a **U.S.-based** energy company primarily involved in **nuclear power**.
- **`RTX`**: Denotes **Raytheon Technologies**, an American leader in **aerospace and defense** sectors.
- **`8031.T`**: Represents **Mitsui & Co.**, a diversified **Japanese conglomerate**, listed on the **Tokyo Stock Exchange**,

In [5]:
# Constants
TICKERS = ['ML.PA', 'HDB', 'CEG', 'RTX', '8031.T']
START_DATE = "2019-01-01"
END_DATE = datetime.today().strftime('%Y-%m-%d')
RISK_FREE_RATE = 0.05  # Risk-free rate as a decimal (5%)

# Code Explanation: - Logging Configuration

This code sets up the logging configuration for the program, ensuring that log messages are captured and displayed according to the defined settings.

## 1. Basic Configuration
The `logging.basicConfig` method configures the logging system with the following parame


```python
logging.basicConfig(
    level=logging.INFO,  # Set the minimum severity level to INFO
    format='%(asctime)s - %(levelname)s - %(message)s',  # Customize log message format
    handlers=[
        logging.FileHandler("portfolio_optimization.log"),  # Log to a file
        logging.StreamHandler()  # Log to console (optional)
    ]
)


In [7]:
logging.basicConfig(
    level=logging.INFO,  # Set the minimum severity level to INFO
    format='%(asctime)s - %(levelname)s - %(message)s',  # Customize log message format
    handlers=[
        logging.FileHandler("portfolio_optimization.log"),  # Log to a file
        logging.StreamHandler()  # Log to console (optional)
    ]
)

# Code Explanation: - Create Config File Function

This function checks for the existence of a configuration file and creates it with default parameters if it doesn't already exist.

## 1. Function Definition
The function `create_config_file` is defined with the following parameter:

```python
def create_config_file(file_path="config.json"):


In [9]:
def create_config_file(file_path="config.json"):
    #Default parameters to include in the config.json file
    default_config = {
        "tickers": ["ML.PA", "HDB", "CEG","RTX","8031.T"],  
        "start_date": "2019-01-01",           
        "end_date": None,                     
        "risk_free_rate": 0.05,               
        "output_path": "./plots/",            
        "base_currency": "USD"                
    }
    if not os.path.exists(file_path): #checks if config.json exists and creates a fresh one if needed
        with open(file_path, "w") as file:
            json.dump(default_config, file, indent=4)
        print(f"Configuration file created at: {os.path.abspath(file_path)}")
        print("Modify this file to update tickers, dates, or other parameters.")
    else:
        print(f"Configuration file already exists at: {os.path.abspath(file_path)}")

# Code Explanation: - Load Configuration Function

This function loads configuration parameters from a JSON file and handles optional or dynamic values.

## 1. Function Definition
The function `load_config` is defined with the following parameter:

```python
def load_config(config_file="config.json"):


In [11]:
def load_config(config_file="config.json"):
    """
    Loads configuration parameters from a JSON file.

    Args:
        config_file (str, optional): Path to the JSON configuration file. Defaults to "config.json".

    Returns:
        dict: Dictionary containing configuration parameters.
    """
    try:
        with open(config_file, "r") as file:
            config = json.load(file)

        # Handle optional or dynamic values
        if config["end_date"] is None:
            config["end_date"] = datetime.today().strftime('%Y-%m-%d')

        return config
    except FileNotFoundError:
        raise FileNotFoundError(f"Configuration file '{config_file}' not found.")
    except json.JSONDecodeError as e:
        raise ValueError(f"Error parsing JSON configuration file: {e}")

create_config_file()
config = load_config()

Configuration file already exists at: C:\Users\louis\config.json


# Code Explanation: - Remove Duplicates Function

This function removes duplicate strings from a list and returns a list of unique strings.

## 1. Function Definition
The function `remove_duplicates` is defined with the following parameter:

```python
def remove_duplicates(strings: list[str]) -> list[str]:


In [13]:
def remove_duplicates(strings: list[str]) -> list[str]:
    seen = set()
    unique_strings = []
    for string in strings:
        if string not in seen:
            unique_strings.append(string)
            seen.add(string)
    return unique_strings

# Constants
TICKERS = remove_duplicates(config["tickers"])
START_DATE = config["start_date"]
END_DATE = config["end_date"]
RISK_FREE_RATE = config["risk_free_rate"]
PATH_TO_PLOTS = config["output_path"]
BASE_CURRENCY = config["base_currency"]

# Code Explanation: - Check or Create Plots Folder

This function checks if a folder exists and creates it if it does not. It ensures that a valid directory is present for storing plot files.

## 1. Function Definition
The function `check_or_create_plots_folder` is defined with the following parameter:

```python
def check_or_create_plots_folder(folder="plots"):


In [15]:
def check_or_create_plots_folder(folder="plots"):
    if os.path.exists(folder):
        if os.path.isdir(folder):
            print(f"Folder '{folder}' already exists.")
        else:
            print(f"A file named '{folder}' exists, but it's not a folder. Please rename or remove it.")
    else:
        os.makedirs(folder)
        print(f"Folder '{folder}' created successfully.")

check_or_create_plots_folder(PATH_TO_PLOTS)
import sys
import subprocess

Folder './plots/' already exists.


# Code Explanation: - Install Required Packages

This function ensures that all necessary Python packages are installed for the script to run. If any package is missing, it will be installed automatically. Additionally, it checks the Python version to ensure compatibility.

## 1. Function Definition
The function `install_required_packages` is defined as follows:

```python
def install_required_packages():


In [17]:
def install_required_packages():
    """
    Installs required Python packages for the script if they are not already installed.
    
    Raises:
        RuntimeError: If the Python version is lower than 3.11.
    """
    required_packages = ['yfinance', 'numpy', 'pandas', 'scipy', 'matplotlib', 'seaborn', 'statsmodels']
    for package in required_packages:
        try:
            __import__(package)
        except ImportError:
            print(f"Installing {package}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
    if sys.version_info < (3, 11):
        raise RuntimeError("This script requires Python 3.11 or newer")

install_required_packages()

# Code Explanation: - Utility Function for User Input

This function prompts the user to provide input and returns the response, using a default value if no input is given.

## 1. Function Definition
The function `answer_input` is defined as follows:

```python
def answer_input(prompt, default="yes"):


In [19]:
# Utility Functions
def answer_input(prompt, default="yes"):
    """
    Prompts the user for input with a default value.

    Args:
        prompt (str): The question or message displayed to the user.
        default (str, optional): The default value returned if no input is provided. Defaults to "yes".

    Returns:
        str: The user's response or the default value.
    """
    print(prompt)
    return input().strip().lower() or default

# Code Explanation: - Fetch Stock Data Function

This function retrieves historical adjusted closing prices for a given stock within a specified date range.

## 1. Function Definition
The function `fetch_stock_data` is defined with the following parameters:

```python
def fetch_stock_data(ticker, start_date, end_date):


In [21]:
def fetch_stock_data(ticker, start_date, end_date):
    """
    Fetches historical adjusted closing prices for a specific stock.

    Args:
        ticker (str): The stock's ticker symbol.
        start_date (str): The start date for fetching data in 'YYYY-MM-DD' format.
        end_date (str): The end date for fetching data in 'YYYY-MM-DD' format.

    Returns:
        pd.Series: Time series of adjusted closing prices.
        None: If no data is available or an error occurs.
    """
    try:
        data = yf.download(ticker, start=start_date, end=end_date, progress=False)['Adj Close']
        return data if not data.empty else None
    except Exception as e:
        logging.error(f"Error fetching data for {ticker}: {e}")
        return None

# Code Explanation: - Get Company Name Function

This function retrieves the company's short name for a given stock ticker. If the name is not available or an error occurs, it returns the ticker itself.

## 1. Function Definition
The function `get_company_name` is defined with the following parameter:

```python
def get_company_name(ticker):


In [23]:
def get_company_name(ticker):
    """
    Retrieves the company's short name for a given stock ticker.

    Args:
        ticker (str): The stock's ticker symbol.

    Returns:
        str: The company's short name, or the ticker if the name cannot be retrieved.
    """
    try:
        return yf.Ticker(ticker).info.get('shortName', ticker)
    except Exception:
        logging.error(f'Error running get_company_name for {ticker}')
        return ticker


# Code Explanation: - Check Tickers and Fetch Data Function

This function validates and fetches historical stock data for a list of tickers, creating a combined DataFrame with adjusted closing prices and returning relevant information.

## 1. Function Definition
The function `check_tickers_fetching` is defined with the following parameters:

```python
def check_tickers_fetching(tickers, start_date, end_date):


In [25]:
def check_tickers_fetching(tickers, start_date, end_date):
    """
    Validates and fetches historical data for a list of stock tickers.

    Args:
        tickers (list of str): List of stock ticker symbols.
        start_date (str): The start date for fetching data in 'YYYY-MM-DD' format.
        end_date (str): The end date for fetching data in 'YYYY-MM-DD' format.

    Returns:
        tuple:
            - list of str: Valid stock tickers with available data.
            - dict: Dictionary mapping tickers to their time series data.
            - list of str: Company names corresponding to valid tickers.
            - pd.DataFrame: Combined DataFrame of all valid tickers' adjusted closing prices.

    Raises:
        ValueError: If no valid data is downloaded for the provided tickers.
    """
    valid_tickers = []
    data_dict = {}
    company_names = []

    for ticker in tickers:# First, collect all valid data
        data = fetch_stock_data(ticker, start_date, end_date)
        if data is not None:
            valid_tickers.append(ticker)
            data_dict[ticker] = data
            company_names.append(get_company_name(ticker))
    if not valid_tickers:
        raise ValueError("No valid data downloaded for the provided tickers.")
   
    data = data_dict[valid_tickers[0]] # Create DataFrame from the first valid ticker's data
    
    for ticker in valid_tickers[1:]:# Join other tickers' data
        data = pd.concat([data, data_dict[ticker]], axis=1)
   
    data.columns = valid_tickers # Set column names
    return valid_tickers, data_dict, company_names, data

# Code Explanation: - Filter Extreme Values Function

This function filters out stocks from a DataFrame based on their annual returns and volatilities, excluding those that meet the criteria for "extreme" values.

## 1. Function Definition
The function `filter_extreme_values` is defined as follows:

```python
def filter_extreme_values(metrics_df, SPREAD=1.8):


In [27]:
def filter_extreme_values(metrics_df, SPREAD=1.8):
    """
    Filters out stocks with extreme annual returns or volatilities.

    Args:
        metrics_df (pd.DataFrame): DataFrame containing annual return and volatility metrics.
        SPREAD (float, optional): Multiplier for the mean to determine extreme values. Defaults to 1.8.

    Returns:
        pd.DataFrame: Filtered DataFrame with extreme values excluded.
    """
    return_threshold = metrics_df["Annual_Return%"].mean() * SPREAD
    volatility_threshold = metrics_df["Annual_Volatility%"].mean() * SPREAD
    
    is_extreme = (metrics_df["Annual_Return%"].abs() > return_threshold) | (metrics_df["Annual_Volatility%"] > volatility_threshold)
    
    extreme_stocks = metrics_df[is_extreme]
    for _, row in extreme_stocks.iterrows():
        print(f"\nTicker {row['Stock_Ticker']} has extreme values: Return={row['Annual_Return%']:.2f}%, Volatility={row['Annual_Volatility%']:.2f}%")
        if answer_input("Do you want to include this stock in the analysis? (yes/no): ") != "yes":
            print(f"\nExcluding {row['Stock_Ticker']} from the analysis.")
            metrics_df = metrics_df[metrics_df.Stock_Ticker != row['Stock_Ticker']]
    return metrics_df

# Code Explanation: - Compute Stock Metrics Function

This function calculates key financial metrics for a list of stock tickers, including annual returns, annual volatility, and correlation/covariance matrices.

## 1. Function Definition
The function `compute_stock_metrics` is defined as follows:

```python
def compute_stock_metrics(tickers, start_date, end_date):


In [29]:
def compute_stock_metrics(tickers, start_date, end_date):
    """
    Computes financial metrics such as annual return and volatility for a list of stocks.

    Args:
        tickers (list of str): List of stock ticker symbols.
        start_date (str): The start date for fetching data in 'YYYY-MM-DD' format.
        end_date (str): The end date for fetching data in 'YYYY-MM-DD' format.

    Returns:
        dict: A dictionary containing:
            - "Metrics_DataFrame" (pd.DataFrame): Annual return and volatility metrics.
            - "Correlation_Matrix" (pd.DataFrame): Correlation matrix of daily returns.
            - "Covariance_Matrix" (pd.DataFrame): Covariance matrix of daily returns.
            - "Raw Data" (pd.DataFrame): Adjusted closing prices of the valid stocks.
    """
    valid_tickers, data_dict, company_names, data = check_tickers_fetching(tickers, start_date, end_date)
    daily_returns = data.pct_change()
    
    metrics_df = pd.DataFrame({
        "Stock_Ticker": valid_tickers,
        "Company_Name": company_names,
        "Annual_Return%": daily_returns.mean() * 252 * 100,
        "Annual_Volatility%": daily_returns.std() * np.sqrt(252) * 100
    })
    
    metrics_df = filter_extreme_values(metrics_df) # Apply filter_extreme_values
    
    metrics_df = metrics_df.dropna() # Remove rows with any NaN values
    
    # Recalculate correlation and covariance matrices with clean data
    clean_daily_returns = daily_returns[metrics_df["Stock_Ticker"]]
    correlation_matrix = clean_daily_returns.corr()
    cov_matrix = clean_daily_returns.cov() * 252
    
    return {
        "Metrics_DataFrame": metrics_df,
        "Correlation_Matrix": correlation_matrix,
        "Covariance_Matrix": cov_matrix,
        "Raw Data": data[metrics_df["Stock_Ticker"]]
    }

# Code Explanation: - Print Portfolio Weights Function

This function prints the portfolio weights in a readable tabular format, showing each stock ticker and its respective weight as a percentage.

## 1. Function Definition
The function `print_portfolio_weights` is defined as follows:

```python
def print_portfolio_weights(weights, tickers):


In [31]:
# Function to print portfolio weights in a readable format
def print_portfolio_weights(weights, tickers):
    """
    Prints portfolio weights in a tabular format.

    Args:
        weights (list of float): Portfolio weights for each stock.
        tickers (list of str): Stock ticker symbols corresponding to the weights.
    """
    print("\nPortfolio Weights:")
    print(f"{'Ticker':<10}{'Weight (%)':<15}")
    print("-" * 25)
    for ticker, weight in zip(tickers, weights):
        print(f"{ticker:<10}{weight * 100:<15.2f}")
        

# Code Explanation: - Portfolio Metrics Calculation Function

This function calculates key performance metrics for a portfolio, including the portfolio's expected return, volatility, and Sharpe ratio.

## 1. Function Definition
The function `portfolio_metrics` is defined as follows:

```python
def portfolio_metrics(weights, returns, cov_matrix):


In [33]:
# Portfolio Optimization Functions
def portfolio_metrics(weights, returns, cov_matrix):
    """
    Calculates portfolio return, volatility, and Sharpe ratio.

    Args:
        weights (np.ndarray): Portfolio weights.
        returns (np.ndarray): Annualized returns for each stock.
        cov_matrix (np.ndarray): Covariance matrix of stock returns.

    Returns:
        tuple:
            - float: Portfolio return.
            - float: Portfolio volatility.
            - float: Sharpe ratio of the portfolio.
    """
    portfolio_return = np.dot(weights, returns)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe_ratio = (portfolio_return - RISK_FREE_RATE) / portfolio_volatility
    return portfolio_return, portfolio_volatility, sharpe_ratio

# Code Explanation: - Portfolio Optimization Function

This function optimizes portfolio weights based on the selected optimization type, either finding the Global Minimum Variance Portfolio (GMVP) or the Maximum Sharpe Ratio (MSR) portfolio.

## 1. Function Definition
The function `optimize_portfolio` is defined as follows:

```python
def optimize_portfolio(mean_returns, cov_matrix, opt_type):


In [35]:
def optimize_portfolio(mean_returns, cov_matrix, opt_type):
    """
    Optimizes portfolio weights based on the chosen optimization type.

    Args:
        mean_returns (np.ndarray): Annualized mean returns for each stock.
        cov_matrix (np.ndarray): Covariance matrix of stock returns.
        opt_type (str): Optimization type ("GMVP" for Global Minimum Variance Portfolio or "MSR" for Maximum Sharpe Ratio).

    Returns:
        tuple:
            - np.ndarray: Optimized portfolio weights.
            - float: Objective function value (e.g., portfolio volatility for GMVP).
    """
    num_assets = len(mean_returns)
    constraints = {"type": "eq", "fun": lambda x: np.sum(x) - 1}
    bounds = tuple((0.0, 1.0) for _ in range(num_assets))
    
    if opt_type == "GMVP":
        objective = lambda weights: np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    else:
        objective = lambda weights: -(np.dot(weights, mean_returns) - RISK_FREE_RATE) / np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    
    result = sco.minimize(objective, num_assets * [1.0 / num_assets], method='SLSQP', bounds=bounds, constraints=constraints)
    return result.x, result.fun

# Code Explanation: - Efficient Frontier Calculation Function

This function calculates the efficient frontier for a portfolio by varying the target return and finding the corresponding portfolio volatility for each.

## 1. Function Definition
The function `calculate_efficient_frontier` is defined as follows:

```python
def calculate_efficient_frontier(mean_returns, cov_matrix, num_points=100):



In [37]:
def calculate_efficient_frontier(mean_returns, cov_matrix, num_points=100):
    """
    Computes the efficient frontier by varying target returns.

    Args:
        mean_returns (np.ndarray): Annualized mean returns for each stock.
        cov_matrix (np.ndarray): Covariance matrix of stock returns.
        num_points (int, optional): Number of points to compute on the efficient frontier. Defaults to 100.

    Returns:
        tuple:
            - np.ndarray: Target returns.
            - np.ndarray: Corresponding portfolio volatilities.
    """
    target_returns = np.linspace(mean_returns.min(), mean_returns.max(), num_points)
    efficient_volatilities = []
    
    for target_return in target_returns:
        constraints = (
            {"type": "eq", "fun": lambda weights: np.sum(weights) - 1},
            {"type": "eq", "fun": lambda weights: np.dot(weights, mean_returns) - target_return}
        )
        bounds = [(0.0, 1.0) for _ in range(len(mean_returns))]
        result = sco.minimize(
            lambda weights: np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))),
            np.ones(len(mean_returns)) / len(mean_returns),
            method='SLSQP',
            bounds=bounds,
            constraints=constraints
        )
        efficient_volatilities.append(result.fun * 100 if result.success else np.nan)
    
    target_returns = target_returns[~np.isnan(efficient_volatilities)]
    efficient_volatilities = np.array(efficient_volatilities)[~np.isnan(efficient_volatilities)]
    return target_returns * 100, efficient_volatilities

# Code Explanation: - Monte Carlo Portfolio Simulation Function

This function simulates a specified number of random portfolios to analyze potential portfolio returns, volatilities, and Sharpe ratios using a Monte Carlo approach.

## 1. Function Definition
The function `simulate_portfolios` is defined as:

```python
def simulate_portfolios(returns, cov_matrix, num_portfolios=2000):


In [39]:
def simulate_portfolios(returns, cov_matrix, num_portfolios=2000):
    """
    Simulates random portfolios for Monte Carlo analysis.

    Args:
        returns (np.ndarray): Annualized mean returns for each stock.
        cov_matrix (np.ndarray): Covariance matrix of stock returns.
        num_portfolios (int, optional): Number of random portfolios to simulate. Defaults to 2000.

    Returns:
        tuple:
            - np.ndarray: Simulated portfolio returns.
            - np.ndarray: Simulated portfolio volatilities.
            - np.ndarray: Simulated Sharpe ratios.
    """
    num_assets = len(returns)
    weights = np.random.rand(num_portfolios, num_assets)
    weights /= np.sum(weights, axis=1)[:, np.newaxis]

    portfolio_returns = np.dot(weights, returns) * 100
    portfolio_volatilities = np.sqrt(np.einsum('ij,jk,ik->i', weights, cov_matrix, weights)) * 100
    sharpe_ratios = (portfolio_returns - RISK_FREE_RATE * 100) / portfolio_volatilities

    return portfolio_returns, portfolio_volatilities, sharpe_ratios

# Code Explanation: - Plot Creation Function

The `create_plot` function is a utility function that sets up a Matplotlib plot with a specific style and basic configurations. It is designed to streamline the creation of plots by setting up commonly used parameters in one place.

## 1. Function Definition
The function `create_plot` is defined as:

```python
def create_plot(figsize=(12, 8), title="", xlabel="", ylabel="", dpi=300):


In [41]:
def create_plot(figsize=(12, 8), title="", xlabel="", ylabel="", dpi=300):
    """
    Creates a styled Matplotlib plot with basic configurations.

    Args:
        figsize (tuple, optional): Figure dimensions (width, height). Defaults to (12, 8).
        title (str, optional): Title of the plot. Defaults to "".
        xlabel (str, optional): Label for the X-axis. Defaults to "".
        ylabel (str, optional): Label for the Y-axis. Defaults to "".
        dpi (int, optional): Resolution of the plot. Defaults to 300.

    Returns:
        matplotlib.axes.Axes: The configured axes object.
    """
    plt.figure(figsize=figsize, dpi=dpi)
    sns.set_style("whitegrid")
    plt.title(title, fontsize=16, pad=20)
    plt.xlabel(xlabel, fontsize=14)
    plt.ylabel(ylabel, fontsize=14)
    plt.grid(alpha=0.3)
    return plt.gca()

# Code Explanation: - Plot Finalization Function

The `finalize_plot` function is designed to enhance and save a Matplotlib plot. It provides flexibility to include legends, adjust layout, and display the plot interactively.

## 1. Function Definition
The function `finalize_plot` is defined as:

```python
def finalize_plot(ax, filename, legend=True, legend_title=None, tight_layout=True, show=True):


In [43]:
def finalize_plot(ax, filename, legend=True, legend_title=None, tight_layout=True, show=True):
    """
    Finalizes a Matplotlib plot by adding legends, saving to file, and optionally displaying it.

    Args:
        ax (matplotlib.axes.Axes): The plot's axes object.
        filename (str): Filename to save the plot.
        legend (bool, optional): Whether to include a legend. Defaults to True.
        legend_title (str, optional): Title for the legend. Defaults to None.
        tight_layout (bool, optional): Whether to apply a tight layout. Defaults to True.
        show (bool, optional): Whether to display the plot. Defaults to True.
    """
    if legend:
        if legend_title:
            ax.legend(title=legend_title, fontsize=10, loc="best", frameon=True, fancybox=True, framealpha=0.6)
        else:
            ax.legend(fontsize=10, loc="best", frameon=True, fancybox=True, framealpha=0.6)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(f'{PATH_TO_PLOTS}{filename}.png')
    plt.show()

# Code Explanation: - Fetch and Convert Stock Data Function

The `fetch_and_convert_data` function is designed to download historical stock price data for a list of tickers and convert the prices into a specified base currency.

## 1. Function Definition
The function `fetch_and_convert_data` is defined as:

```python
def fetch_and_convert_data(tickers, start_date, end_date, base_currency="USD"):


In [45]:
def fetch_and_convert_data(tickers, start_date, end_date, base_currency="USD"):
    """
    Fetches historical stock prices and converts them to a specified base currency.

    Args:
        tickers (list of str): List of stock ticker symbols.
        start_date (str): Start date for fetching data in 'YYYY-MM-DD' format.
        end_date (str): End date for fetching data in 'YYYY-MM-DD' format.
        base_currency (str, optional): Target currency for conversion. Defaults to "USD".

    Returns:
        dict: A dictionary where keys are tickers and values are time series of converted prices.
    """
    data_dict = {}
    conversion_rates = {"USD": 1.0, "EUR": 1.1, "JPY": 0.008, "GBP": 1.25}
    
    for ticker in tickers:
        try:
            stock_info = yf.Ticker(ticker).info
            stock_currency = stock_info.get('currency', 'Unknown')
            
            data = yf.download(ticker, start=start_date, end=end_date, progress=False)['Adj Close']
            
            if not data.empty and stock_currency in conversion_rates:
                data = data * conversion_rates[stock_currency]
                data_dict[ticker] = data
            else:
                print(f"No data or missing conversion rate for {ticker}. Skipping...")
        except Exception as e:
            logging.error(f"Error processing {ticker}: {e}") 
    return data_dict

# Code Explanation: - Visualization Function for Historical Prices

The `plot_historical_prices` function generates a line plot to visualize the historical stock prices of specified tickers, converted into a target currency.

## 1. Function Definition
The function `plot_historical_prices` is defined as:

```python
def plot_historical_prices(metrics_df, start_date, end_date, base_currency="USD", title="Historical Prices (Converted)"):


In [47]:
# Visualization Functions
def plot_historical_prices(metrics_df, start_date, end_date, base_currency="USD", title="Historical Prices (Converted)"):
    """
    Plots historical stock prices converted to a specified currency.

    Args:
        metrics_df (pd.DataFrame): DataFrame containing stock tickers and company names.
        start_date (str): Start date for fetching data in 'YYYY-MM-DD' format.
        end_date (str): End date for fetching data in 'YYYY-MM-DD' format.
        base_currency (str, optional): Target currency for price conversion. Defaults to "USD".
        title (str, optional): Title of the plot. Defaults to "Historical Prices (Converted)".
    """
    ax = create_plot(figsize=(14, 8), title=title, xlabel="Date", ylabel=f"Adjusted Price ({base_currency})")
    
    tickers = metrics_df['Stock_Ticker'].tolist()
    company_names = metrics_df['Company_Name'].tolist()
    data_dict = fetch_and_convert_data(tickers, start_date, end_date, base_currency)
    
    for ticker, name in zip(tickers, company_names):
        if ticker in data_dict:
            ax.plot(data_dict[ticker].index, data_dict[ticker], label=f"{name} ({ticker})", linewidth=2)
    
    plt.figtext(0.5, -0.05, f"Note: Prices are converted to {base_currency} using static exchange rates; stocks with no data are skipped.",
                wrap=True, horizontalalignment='center', fontsize=10, color="gray")
    
    finalize_plot(ax, "Historical_prices", legend_title="Stocks")

# Code Explanation: - Visualization Function for Correlation Heatmap

The `plot_correlation_heatmap` function generates a heatmap to visually represent the correlation matrix of stock returns.

## 1. Function Definition
The function `plot_correlation_heatmap` is defined as:

```python
def plot_correlation_heatmap(correlation_matrix, tickers, title="Correlation Heatmap"):


In [49]:
def plot_correlation_heatmap(correlation_matrix, tickers, title="Correlation Heatmap"):
    """
    Plots a heatmap of the correlation matrix for the given stocks.

    Args:
        correlation_matrix (pd.DataFrame): Correlation matrix of stock returns.
        tickers (list of str): Stock ticker symbols.
        title (str, optional): Title of the plot. Defaults to "Correlation Heatmap".
    """
    ax = create_plot(figsize=(10, 8), title=title)
    sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f",
                xticklabels=tickers, yticklabels=tickers, linewidths=0.5,
                cbar_kws={'label': 'Correlation Coefficient'}, ax=ax)
    plt.xticks(rotation=45, ha='right', fontsize=10)
    plt.yticks(rotation=0, fontsize=10)
    finalize_plot(ax, "Correlation_heatmap", legend=False)

# Code Explanation: - Visualization Function for GMVP and MSR Portfolio Weights

The `plot_gmvp_msr_weights` function creates a bar plot to visualize the portfolio weights of the Global Minimum Variance Portfolio (GMVP) and Maximum Sharpe Ratio Portfolio (MSR).

## 1. Function Definition
The function `plot_gmvp_msr_weights` is defined as:

```python
def plot_gmvp_msr_weights(tickers, company_names, gmvp_weights, msr_weights, gmvp_performance, msr_performance):


In [51]:
def plot_gmvp_msr_weights(tickers, company_names, gmvp_weights, msr_weights, gmvp_performance, msr_performance):
    """
    Visualizes the portfolio weights for GMVP and MSR optimization.

    Args:
        tickers (list of str): List of stock ticker symbols.
        company_names (list of str): List of corresponding company names.
        gmvp_weights (np.ndarray): Optimized weights for the GMVP portfolio.
        msr_weights (np.ndarray): Optimized weights for the MSR portfolio.
        gmvp_performance (tuple): Performance metrics for GMVP (return, volatility).
        msr_performance (tuple): Performance metrics for MSR (return, volatility).
    """
    ax = create_plot(figsize=(12, 8), title="Portfolio Weights for GMVP and MSR", xlabel="Stocks", ylabel="Weight (%)")
    
    labels = [f"{name} ({ticker})" for name, ticker in zip(company_names, tickers)]
    x = np.arange(len(labels))
    width = 0.1
    offset = 0.15
    
    bars_gmvp = ax.bar(x - offset, gmvp_weights * 100, width, label='GMVP Weights', color='red', edgecolor='black')
    bars_msr = ax.bar(x + offset, msr_weights * 100, width, label='MSR Weights', color='blue', edgecolor='black')
    
    for bars, weights in [(bars_gmvp, gmvp_weights), (bars_msr, msr_weights)]:
        for bar, weight in zip(bars, weights):
            ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 1,
                    f"{weight * 100:.1f}%", ha='center', va='bottom', fontsize=10, color='black')
    
    gmvp_return, gmvp_volatility = gmvp_performance
    msr_return, msr_volatility = msr_performance
    
    ax.text(len(labels) - 1, max(max(gmvp_weights), max(msr_weights)) * 100 - 15,
            f"GMVP:\nReturn={gmvp_return * 100:.2f}%,\nVolatility={gmvp_volatility * 100:.2f}%",
            color="red", fontsize=8, ha='center',
            bbox=dict(facecolor="white", alpha=0.4, edgecolor="gray"))
    
    ax.text(len(labels) - 1, max(max(gmvp_weights), max(msr_weights)) * 100 - 25,
            f"MSR:\nReturn={msr_return * 100:.2f}%,\nVolatility={msr_volatility * 100:.2f}%",
            color="blue", fontsize=8, ha='center',
            bbox=dict(facecolor="white", alpha=0.4, edgecolor="gray"))
    
    ax.set_xticks(x)
    ax.set_xticklabels(labels, fontsize=10, rotation=45, ha='right')
    finalize_plot(ax,"Gmvp_msr_weights")

# Code Explanation: - Visualization Function for Return vs Risk of Portfolios and Assets

The `plot_return_vs_risk_portfolio_GMVP_MSR` function generates a plot showing the risk-return profile of individual assets, simulated portfolios, and specific optimized portfolios such as the GMVP (Global Minimum Variance Portfolio) and MSR (Maximum Sharpe Ratio Portfolio). It also includes the efficient frontier and the Capital Market Line (CML).

## 1. Function Definition
The function `plot_return_vs_risk_portfolio_GMVP_MSR` is defined as:

```python
def plot_return_vs_risk_portfolio_GMVP_MSR(stock_df, simulated_returns, simulated_volatilities, simulated_sharpe_ratios,
                                           efficient_returns, efficient_volatilities, msr_return, msr_volatility,
                                           min_risk_return, min_risk_volatility):


In [53]:
def plot_return_vs_risk_portfolio_GMVP_MSR(stock_df, simulated_returns, simulated_volatilities, simulated_sharpe_ratios,
                                           efficient_returns, efficient_volatilities, msr_return, msr_volatility,
                                           min_risk_return, min_risk_volatility):
    """
    Plots the risk-return characteristics of the portfolio and individual assets.

    Args:
        stock_df (pd.DataFrame): DataFrame containing stock return and volatility metrics.
        simulated_returns (np.ndarray): Simulated portfolio returns.
        simulated_volatilities (np.ndarray): Simulated portfolio volatilities.
        simulated_sharpe_ratios (np.ndarray): Simulated Sharpe ratios.
        efficient_returns (np.ndarray): Returns on the efficient frontier.
        efficient_volatilities (np.ndarray): Volatilities on the efficient frontier.
        msr_return (float): Return of the Maximum Sharpe Ratio portfolio.
        msr_volatility (float): Volatility of the Maximum Sharpe Ratio portfolio.
        min_risk_return (float): Return of the Global Minimum Variance Portfolio.
        min_risk_volatility (float): Volatility of the Global Minimum Variance Portfolio.
    """
    ax = create_plot(figsize=(14, 10), title="Expected Return vs Risk for Portfolio and Individual Assets",
                     xlabel="Risk (Volatility %)", ylabel="Expected Return (%)")
    
    scatter = ax.scatter(simulated_volatilities, simulated_returns, c=simulated_sharpe_ratios, cmap="viridis", alpha=0.6, s=10)
    plt.colorbar(scatter, label="Sharpe Ratio", shrink=1, aspect=30, pad=0.02)
    
    ax.scatter(0, RISK_FREE_RATE * 100, color="lime", label="Risk-Free Rate", edgecolor="Teal", s=150, zorder=3)
    ax.text(-0.2, RISK_FREE_RATE * 100 + 8, "Risk-Free\n(5%)", fontsize=10, ha="center",
            bbox=dict(facecolor="white", edgecolor="gray", boxstyle="round,pad=0.3"))
    
    for _, row in stock_df.iterrows():
        ax.scatter(row["Stock_volatility%"], row["Stock_return%"], color="orange", edgecolor="black", s=100, zorder=3)
        ax.text(row["Stock_volatility%"] + 0.5, row["Stock_return%"] + 2,
                f"{row['Stock_name']}\n({row['Stock_return%']:.1f}%)", fontsize=9,
                bbox=dict(facecolor="white", edgecolor="gray", boxstyle="round,pad=0.3"))
        ax.plot([row["Stock_volatility%"], min_risk_volatility * 100],
                [row["Stock_return%"], min_risk_return * 100], linestyle="dotted", color="gray")
    
    ax.scatter(min_risk_volatility * 100, min_risk_return * 100, color="red", edgecolor="gray",
               label="GMVP (Minimum Risk)", marker="H", s=180, zorder=3)
    ax.text(min_risk_volatility * 100 - 3.2, min_risk_return * 100 + 2,
            f"GMVP\n({min_risk_return * 100:.2f}%)", fontsize=10,
            bbox=dict(facecolor="white", edgecolor="gray", boxstyle="round,pad=0.3"))
    
    ax.scatter(msr_volatility * 100, msr_return * 100, color="blue",
               label="MSR (Maximum Sharpe Ratio) Portfolio", marker="o", s=180, zorder=3)
    ax.text(msr_volatility * 100 - 3, msr_return * 100 + 5,
            f"MSR\n({msr_return * 100:.2f}%)", fontsize=10,
            bbox=dict(facecolor="white", edgecolor="gray", boxstyle="round,pad=0.3"))
    
    ax.plot(efficient_volatilities, efficient_returns, color="blue", linestyle="--", linewidth=1.5, label="Efficient Frontier")
    
    ax.set_xticks(np.arange(0, max(efficient_volatilities) + 1, 2))
    ax.set_yticks(np.arange(0, max(efficient_returns) + 1, 5))
    
    cml_x = np.linspace(0, msr_volatility * 100, 100)
    cml_y = RISK_FREE_RATE * 100 + (msr_return * 100 - RISK_FREE_RATE * 100) * (cml_x / (msr_volatility * 100))
    ax.plot(cml_x, cml_y, label="Capital Market Line (CML)", color="green", linestyle="-", linewidth=1.5)
    
    finalize_plot(ax, "Return_vs_risk_portfolio_GMVP_MSR")

# Code Explanation: - Fetching Fama-French 3-Factor Data

The `fetch_fama_french_factors` function retrieves the daily data for the Fama-French 3-factor model, which includes market excess returns, size premium, and value premium. It also includes the risk-free rate.

## 1. Function Definition
The function `fetch_fama_french_factors` is defined as:

```python
def fetch_fama_french_factors(start_date, end_date):


In [55]:
def fetch_fama_french_factors(start_date, end_date):
    """
    Fetches Fama-French 3-factor data for the specified date range.

    Args:
        start_date (str): Start date for the data in 'YYYY-MM-DD' format.
        end_date (str): End date for the data in 'YYYY-MM-DD' format.

    Returns:
        pd.DataFrame: Daily Fama-French factors (MKT-RF, SMB, HML, RF).
    """
    ff3_data = pdr.DataReader('F-F_Research_Data_Factors_daily', 'famafrench', start_date, end_date)[0]
    ff3_data.index = pd.to_datetime(ff3_data.index)
    ff3_data = ff3_data.rename(columns={'Mkt-RF': 'MKT-RF', 'HML': 'HML', 'SMB': 'SMB', 'RF': 'RF'})
    ff3_data = ff3_data / 100
    return ff3_data

# Code Explanation: - Fama-French 3-Factor Analysis for Portfolio Returns

The `compute_fama_french_3_factors` function performs a regression analysis using the Fama-French 3-factor model on a portfolio's excess returns. It visualizes the coefficients of the regression, compares fitted versus actual returns, and summarizes factor statistics.

## 1. Function Definition
The function `compute_fama_french_3_factors` is defined as:

```python
def compute_fama_french_3_factors(portfolio_returns, ff3_factors, portfolio_name="Portfolio"):


In [57]:
def compute_fama_french_3_factors(portfolio_returns, ff3_factors, portfolio_name="Portfolio"):
    """
    Performs Fama-French 3-factor regression on the portfolio's excess returns.

    Args:
        portfolio_returns (pd.Series): Portfolio returns over time.
        ff3_factors (pd.DataFrame): Fama-French 3-factor data.
        portfolio_name (str, optional): Name of the portfolio. Defaults to "Portfolio".

    Outputs:
        Plots of regression coefficients and fitted vs. actual returns.
    """
    common_dates = portfolio_returns.index.intersection(ff3_factors.index)
    portfolio_returns = portfolio_returns.loc[common_dates]
    ff3_factors = ff3_factors.loc[common_dates]
    portfolio_excess_returns = portfolio_returns - ff3_factors['RF']
    
    X = sm.add_constant(ff3_factors[['MKT-RF', 'SMB', 'HML']])

    model = sm.OLS(portfolio_excess_returns, X).fit()
    #print(model.summary()) #withdraw the '#' before the print if you want to access to OLS Regression Results

    factors = ['Alpha (Intercept)', 'MKT-RF', 'SMB', 'HML']
    coefficients = model.params
    
    ax = create_plot(figsize=(10, 6), title=f"Fama-French 3-Factor Coefficients for {portfolio_name}", 
                     xlabel="Factors", ylabel="Coefficient Value")
    ax.bar(factors, coefficients, color=['blue', 'orange', 'green', 'red'], edgecolor='black')
    finalize_plot(ax, "fama_french_3_fig1", legend=False)

    ax = create_plot(figsize=(10, 6), title=f"Fitted vs Actual Excess Returns for {portfolio_name}", 
                     xlabel="Actual Excess Returns", ylabel="Fitted Excess Returns")
    ax.scatter(portfolio_excess_returns, model.fittedvalues, alpha=0.7, label="Fitted vs Actual")
    ax.plot([portfolio_excess_returns.min(), portfolio_excess_returns.max()], 
            [portfolio_excess_returns.min(), portfolio_excess_returns.max()], 
            color="red", linestyle="--", label="45° Line")
    finalize_plot(ax, "fama_french_3_fig2")

    print("\n\n=== Fama-French Factor Ratios ===\n")
    factor_stats = pd.DataFrame({
        'Mean': ff3_factors.mean(),
        'Std Dev': ff3_factors.std(),
        'Correlation with Portfolio Excess Returns': ff3_factors.corrwith(portfolio_excess_returns)
    }).loc[['MKT-RF', 'SMB', 'HML', 'RF']]
    print(factor_stats)

    ax = create_plot(figsize=(10, 6), title="Fama-French Factor Mean and Standard Deviation", 
                     xlabel="Factors", ylabel="Value")
    factor_stats[['Mean', 'Std Dev']].plot(kind='bar', ax=ax, edgecolor='black')
    finalize_plot(ax, "fama_french_3_fig3")

# Code Explanation: - Plotting the Security Market Line (SML) with GMVP's Beta and Return

The `plot_sml_with_dynamic_gmvp` function creates a visualization of the Security Market Line (SML) and marks the GMVP (Global Minimum Variance Portfolio) on the plot, showing its beta and expected return.

## 1. Function Definition
The function is defined as:
```python
def plot_sml_with_dynamic_gmvp(metrics_df, gmvp_weights, market_ticker='URTH', start_date=START_DATE, end_date=END_DATE):


In [59]:
def plot_sml_with_dynamic_gmvp(metrics_df, gmvp_weights, market_ticker='URTH', start_date=START_DATE, end_date=END_DATE):
    """
    Plots the Security Market Line (SML) with the GMVP's beta and return.

    Args:
        metrics_df (pd.DataFrame): DataFrame containing stock metrics (beta, return, volatility).
        gmvp_weights (np.ndarray): Weights for the GMVP portfolio.
        market_ticker (str, optional): Ticker symbol for the market index. Defaults to 'URTH'.
        start_date (str): Start date for fetching data in 'YYYY-MM-DD' format.
        end_date (str): End date for fetching data in 'YYYY-MM-DD' format.
    """
    try:
        # Download and prepare market data
        market_data = yf.download(market_ticker, start=start_date, end=end_date, progress=False)        
        if market_data.empty:
            raise ValueError("No data found for market ticker.")
        
        market_daily_returns = market_data['Adj Close'].pct_change().dropna()
        market_annual_return = float(((1 + market_daily_returns.mean()) ** 252) - 1)

        # Calculate betas for stocks
        stock_betas = []
        for ticker in metrics_df['Stock_Ticker']:
            stock_data = yf.download(ticker, start=start_date, end=end_date, progress=False)['Adj Close'].pct_change().dropna()
            
            common_dates = stock_data.index.intersection(market_daily_returns.index)
            stock_data = stock_data.loc[common_dates]
            market_returns = market_daily_returns.loc[common_dates]
            
            X = sm.add_constant(market_returns)
            model = sm.OLS(stock_data, X).fit()
            stock_betas.append(model.params[1])
        
        metrics_df['Beta'] = stock_betas
        gmvp_beta = np.dot(gmvp_weights, metrics_df['Beta'])
        gmvp_return = np.dot(gmvp_weights, metrics_df['Annual_Return%']) / 100

        # Prepare data for SML
        betas = np.linspace(-0.5, 2.5, 100)
        sml_returns = RISK_FREE_RATE + betas * (market_annual_return - RISK_FREE_RATE)

        # Plot SML
        ax = create_plot(title="Security Market Line (SML) with GMVP", xlabel="Beta (Systematic Risk)", ylabel="Expected Return (%)")
        ax.plot(betas, sml_returns * 100, label="SML", linestyle="--", color="blue")

        # Plot GMVP point with label
        ax.scatter(gmvp_beta, gmvp_return * 100, color="red", s=100, label="GMVP")
        ax.annotate(f"GMVP ({gmvp_return:.2%})", 
                    (gmvp_beta, gmvp_return * 100), 
                    xytext=(5, 5), 
                    textcoords='offset points', 
                    color='red', 
                    fontsize=8)

        # Plot individual stocks with labels
        for _, row in metrics_df.iterrows():
            ax.scatter(row['Beta'], row['Annual_Return%'], color='orange', s=50)
            ax.annotate(row['Stock_Ticker'], 
                        (row['Beta'], row['Annual_Return%']), 
                        xytext=(5, 5), 
                        textcoords='offset points', 
                        color='black', 
                        fontsize=8)

        ax.axhline(y=RISK_FREE_RATE * 100, color="green", linestyle="-", label="Risk-Free Rate")
        ax.scatter([], [], color='orange', s=50, label='Stocks')# Add a single point for stocks in the legend
        ax.legend(loc='best', fontsize=8)   # Adjust legend
        finalize_plot(ax, "sml_with_dynamic_gmvp")
    except Exception as e:
        logging.error(f"Error in plot_sml_with_dynamic_gmvp: {e}")

# Code Explanation: - Evaluating GMVP Performance

The `evaluate_gmvp_performance` function calculates various performance metrics for the GMVP (Global Minimum Variance Portfolio) using historical data and relevant financial metrics.

## 1. Function Definition
The function is defined as:
```python
def evaluate_gmvp_performance(gmvp_weights, gmvp_return, gmvp_volatility, cov_matrix, risk_free_rate, market_ticker, start_date, end_date, data, tickers):


In [61]:
def evaluate_gmvp_performance(gmvp_weights, gmvp_return, gmvp_volatility, cov_matrix, risk_free_rate, market_ticker, start_date, end_date, data, tickers):
    """
    Evaluates the performance of the GMVP portfolio using various metrics.

    Args:
        gmvp_weights (np.ndarray): Weights of the GMVP portfolio.
        gmvp_return (float): Return of the GMVP portfolio.
        gmvp_volatility (float): Volatility of the GMVP portfolio.
        cov_matrix (np.ndarray): Covariance matrix of stock returns.
        risk_free_rate (float): Risk-free rate used for calculations.
        market_ticker (str): Ticker symbol for the market index.
        start_date (str): Start date for fetching data in 'YYYY-MM-DD' format.
        end_date (str): End date for fetching data in 'YYYY-MM-DD' format.
        data (pd.DataFrame): Raw data of stock prices.
        tickers (list of str): List of stock tickers in the portfolio.

    Returns:
        dict: Performance metrics including Sharpe ratio, beta, Jensen's alpha, and more.
    """
    try:
        market_data = yf.download(market_ticker, start=start_date, end=end_date, progress=False)["Adj Close"]
        if market_data.empty:
            return {"Error": "Market data is empty."}

        market_log_returns = np.log(market_data / market_data.shift(1)).dropna()
        log_returns = np.log(data / data.shift(1)).dropna()

        # Ensure one-dimensionality
        if len(log_returns.shape) > 1:
            log_returns = log_returns.squeeze()
        if len(market_log_returns.shape) > 1:
            market_log_returns = market_log_returns.squeeze()

        aligned_data = log_returns.join(market_log_returns.rename("Market"), how="inner").dropna()

        market_variance = aligned_data["Market"].var()
        if market_variance == 0 or np.isnan(market_variance):
            return {"Error": "Market returns variance is zero or invalid."}

        # Calculate portfolio beta
        betas = aligned_data.cov().loc[tickers, "Market"] / market_variance
        portfolio_beta = np.dot(gmvp_weights, betas)
       
        annualized_market_return = ((1 + market_log_returns.mean()) ** 252) - 1

        annualized_market_volatility = market_log_returns.std() * np.sqrt(252)
        jensens_alpha = gmvp_return - (risk_free_rate + portfolio_beta * (annualized_market_return - risk_free_rate))

        # Compile performance metrics
        performance_metrics = {
            "GMVP Annualized Return (%)": gmvp_return * 100,
            "GMVP Annualized Volatility (%)": gmvp_volatility * 100,
            "GMVP Sharpe Ratio": (gmvp_return - risk_free_rate) / gmvp_volatility,
            "GMVP Beta": portfolio_beta,
            "GMVP Jensen's Alpha": jensens_alpha,
            "Market Annualized Return (%)": annualized_market_return * 100,
            "Market Annualized Volatility (%)": annualized_market_volatility * 100
        }

        return performance_metrics
    except Exception as e:
        logging.error(f"Error in performance evaluation: {e}")
        return {}

# Code Explanation: - Exporting Portfolio Results to Excel

The `export_to_excel` function takes various portfolio data and exports it to an Excel file using the `pandas` library and the `xlsxwriter` engine. This is useful for generating a detailed report containing stock metrics, portfolio weights, and performance metrics.

## 1. Function Definition
The function is defined as:
```python
def export_to_excel(metrics_df, gmvp_weights, msr_weights, gmvp_performance, msr_performance, performance_metrics, filename="portfolio_results.xlsx"):


In [63]:
def export_to_excel(metrics_df, gmvp_weights, msr_weights, gmvp_performance, msr_performance, performance_metrics, filename="portfolio_results.xlsx"):
    """
    Exports portfolio results and metrics to an Excel file.

    Args:
        metrics_df (pd.DataFrame): DataFrame containing stock metrics.
        gmvp_weights (np.ndarray): Weights for the GMVP portfolio.
        msr_weights (np.ndarray): Weights for the MSR portfolio.
        gmvp_performance (tuple): GMVP return and volatility.
        msr_performance (tuple): MSR return and volatility.
        performance_metrics (dict): Performance metrics of the GMVP.
        filename (str, optional): Name of the output Excel file. Defaults to "portfolio_results.xlsx".
    """
    with pd.ExcelWriter(filename, engine="xlsxwriter") as writer:
        # Save stock metrics
        metrics_df.to_excel(writer, sheet_name="Stock Metrics", index=False)

        # Save GMVP and MSR weights
        pd.DataFrame({
            "Ticker": metrics_df["Stock_Ticker"],
            "Company Name": metrics_df["Company_Name"],
            "GMVP Weight (%)": gmvp_weights * 100,
            "MSR Weight (%)": msr_weights * 100,
        }).to_excel(writer, sheet_name="Portfolio Weights", index=False)

        # Save GMVP and MSR performance
        performance_data = {
            "Metric": ["GMVP Return (%)", "GMVP Volatility (%)", "MSR Return (%)", "MSR Volatility (%)"],
            "Value": [gmvp_performance[0] * 100, gmvp_performance[1] * 100, msr_performance[0] * 100, msr_performance[1] * 100],
        }
        pd.DataFrame(performance_data).to_excel(writer, sheet_name="Portfolio Performance", index=False)

        # Save GMVP additional performance metrics
        pd.DataFrame(list(performance_metrics.items()), columns=["Metric", "Value"]).to_excel(writer, sheet_name="Performance Metrics", index=False)

    print(f"\nResults exported to {filename}")

# Code Explanation: - Main Function for Portfolio Optimization and Analysis Workflow

The `main` function orchestrates an end-to-end portfolio optimization and analysis workflow. It covers computing stock metrics, optimizing portfolios (GMVP and MSR), evaluating their performance, generating plots, and exporting results to an Excel file.

## 1. Function Definition
The function is defined as:
```python
def main():


In [ ]:
def main():
    """
    Executes the end-to-end portfolio optimization and analysis workflow:
    - Computes stock metrics.
    - Optimizes portfolios for GMVP and MSR.
    - Evaluates performance and generates plots.
    - Prints key results and metrics.

    Outputs:
        Results and visualizations in the console and saved files.
    """
    try:
        # Step 1: Compute stock metrics
        stock_metrics = compute_stock_metrics(TICKERS, START_DATE, END_DATE)
        metrics_df = stock_metrics["Metrics_DataFrame"]
        correlation_matrix = stock_metrics["Correlation_Matrix"]
        cov_matrix = stock_metrics["Covariance_Matrix"]
        data = stock_metrics['Raw Data']

        if metrics_df.empty:
            print("No valid stocks remaining after filtering.")
            return

        # Extract necessary values
        returns = metrics_df["Annual_Return%"].values / 100
        tickers = metrics_df["Stock_Ticker"].values

        # Step 2: Portfolio Optimization
        gmvp_weights, gmvp_volatility = optimize_portfolio(returns, cov_matrix, "GMVP")
        gmvp_return = np.dot(gmvp_weights, returns)

        msr_weights, _ = optimize_portfolio(returns, cov_matrix, "MSR")
        msr_return, msr_volatility, _ = portfolio_metrics(msr_weights, returns, cov_matrix)

        # Print Basic Portfolio Results
        print("\n\n===== NEOMA WORLD GROWTH PORTFOLIO =====")
        print(f"\n*Data period: {START_DATE} to {END_DATE}\n")
        print("\nTicker    Return (%)     Volatility (%)    Company Name")
        print("-" * 70)
        for ticker, ret, vol, name in zip(
            tickers, returns * 100, metrics_df['Annual_Volatility%'], metrics_df['Company_Name']
        ):
            print(f"{ticker:<10}{ret:<15.2f}{vol:<20.2f}{name:<23}")

        # Print Optimization Results
        print("\n\n=== Portfolio Metrics ===")
        print(f"\nGMVP Return: {gmvp_return * 100:.2f}%, GMVP Volatility: {gmvp_volatility * 100:.2f}%")
        print_portfolio_weights(gmvp_weights, tickers)
        print(f"\n\nMSR Return: {msr_return * 100:.2f}%, MSR Volatility: {msr_volatility * 100:.2f}%")
        print_portfolio_weights(msr_weights, tickers)

        # Step 3: Performance Evaluation
        performance_metrics = evaluate_gmvp_performance(
            gmvp_weights=gmvp_weights,
            gmvp_return=gmvp_return,
            gmvp_volatility=gmvp_volatility,
            cov_matrix=cov_matrix,
            risk_free_rate=RISK_FREE_RATE,
            market_ticker="URTH",
            start_date=START_DATE,
            end_date=END_DATE,
            data=data,
            tickers=tickers,
        )

        print("\n=== GMVP Performance Metrics ===\n")
        for key, value in performance_metrics.items():
            print(f"{key}: {value:.4f}" if isinstance(value, (float, int)) else f"{key}: {value}")

        # Step 4: Generate Plots
        plot_correlation_heatmap(correlation_matrix, tickers, title="Correlation Heatmap for Selected Stocks")

        efficient_returns, efficient_volatilities = calculate_efficient_frontier(returns, cov_matrix)
        simulated_returns, simulated_volatilities, simulated_sharpe_ratios = simulate_portfolios(returns, cov_matrix)

        stock_df = pd.DataFrame({
            "Stock_name": metrics_df['Stock_Ticker'],
            "Stock_return%": metrics_df['Annual_Return%'],
            "Stock_volatility%": metrics_df["Annual_Volatility%"]
        })

        plot_return_vs_risk_portfolio_GMVP_MSR(
            stock_df,
            simulated_returns,
            simulated_volatilities,
            simulated_sharpe_ratios,
            efficient_returns,
            efficient_volatilities,
            msr_return,
            msr_volatility,
            gmvp_return,
            gmvp_volatility,
        )

        plot_gmvp_msr_weights(
            tickers,
            metrics_df['Company_Name'],
            gmvp_weights,
            msr_weights,
            (gmvp_return, gmvp_volatility),
            (msr_return, msr_volatility),
        )

        plot_historical_prices(metrics_df, START_DATE, END_DATE, base_currency="USD", title="Historical Prices in USD")

        # Step 5: Fama-French Analysis
        portfolio_daily_returns = metrics_df['Annual_Return%'] / 252
        portfolio_returns = pd.Series(
            portfolio_daily_returns.values, index=pd.date_range(start=START_DATE, periods=len(portfolio_daily_returns), freq='B')
        )

        ff3_factors = fetch_fama_french_factors(START_DATE, END_DATE)
        compute_fama_french_3_factors(portfolio_returns, ff3_factors, portfolio_name="Sample Portfolio")

        plot_sml_with_dynamic_gmvp(metrics_df, gmvp_weights, start_date=START_DATE, end_date=END_DATE, market_ticker='URTH')
        
        export_to_excel(
        metrics_df=metrics_df,
        gmvp_weights=gmvp_weights,
        msr_weights=msr_weights,
        gmvp_performance=(gmvp_return, gmvp_volatility),
        msr_performance=(msr_return, msr_volatility),
        performance_metrics=performance_metrics,
        filename="portfolio_results.xlsx"
        )

        print(f"\n - Project NEOMA Business School - MSc International Finance, FMRM Track - \n")
    except Exception as e:
        logging.error(f"An error occurred during execution: {e}")

if __name__ == "__main__":
    main()


Ticker CEG has extreme values: Return=69.91%, Volatility=43.22%
Do you want to include this stock in the analysis? (yes/no): 
